# Evaluation Notebook
This notebook converts the GNN prediction into a real dense neural network, then evaluates the dense NN

### Imports

In [ ]:
cd ..

In [ ]:
# Constants
dataset_num = 3
start_num = 600
end_num = 800

num_models = end_num - start_num

In [ ]:
# Downloaded libraries
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [ ]:
# Local files
from utility.graph2model import assign_biases, assign_weights
from datasets.graph_data import NNDataset
from datasets.class_data import ClassificationDataset
from models.graph import TrainerGCN
from models.dense_nets import DenseNet

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

### Training a simple NN

In [ ]:
# Loading model
trainer_gnn = TrainerGCN()
trainer_gnn.load_state_dict(torch.load("./trained_model/gnnmodel"))
trainer_gnn.eval()

In [ ]:
nndataset = NNDataset(root="./")
data = nndataset[start_num:end_num]

In [ ]:
pred = [None] * num_models
for i in range(num_models):
    pred[i] = trainer_gnn(data[i])

In [ ]:
target_model = [None] * num_models
for i in range(num_models):
    target_model[i] = DenseNet(data[i].design)
    assign_biases(target_model[i], pred[i][1], input_size=data[i].design[0])
    assign_weights(target_model[i], pred[i][0])
    target_model[i] = target_model[i].to(device)

### Evaluating the NN

In [ ]:
dataset = ClassificationDataset(f'./raw/dataset_{dataset_num}/')
test_dataloader = DataLoader(dataset, batch_size=32)

In [ ]:
loss_fn = nn.CrossEntropyLoss()

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, hits = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            hits += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()

    acc = hits / size * 100
    test_loss /= num_batches

    return hits, acc, test_loss

In [ ]:
size = len(test_dataloader.dataset)
with torch.no_grad():
    for i in range(num_models):
        hits, acc, test_loss = test(test_dataloader, target_model[i], loss_fn)
        print(f"{i+1:3}-th model  [  Hits : {hits}/{size}  |  Acc : {acc:.3f}  |  loss = {test_loss:.6f}  ]")